# Displaying webcam video in IPython notebook at (relatively) high framerate

When working on my project I realized, I can use OpenCV in Python to grap image from webcam as Numpy array, modify it and then display it using OpenCV's **cv2.imshow()**. OpenCV will create a window and push my frame there. However, this will not work in a IPython notebook. I found few solutions to implement the same functionality, but they all were slow (about 250 ms per frame).

Here, I combine and modify these two examples to get achieve about 5 times higher framerate:
1. __[Showing webcame image using OpenCV and matplotlib](https://gist.github.com/tibaes/35b9dbd7cbf81a98955067aa318290e7#file-video)__. Important moment here is that previous frame is cleared from screen using **IPython.display.clear_output()**
2. __[Minimal code for rendering a numpy array as an image in a Jupyter notebook in memory](https://gist.github.com/kylemcdonald/2f1b9a255993bf9b2629)__. It uses PIL to convert NumPy array to .PNG format in order to display it with **IPython.display.display()**

Both are relatively slow. The slowest step in the first one is the __[matplotlib.pyplot.imshow()](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html)__ and the second one spend most of the time converting array data to PNG in __[PIL.Image.save()](https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.save)__.

But converting to PNG is not the fastest and only give me 2-3 FPS. If I use JPEG instead, framerate goes up to 36 FPS, which is not bad.

In [1]:
# Import the required modules
import cv2
import time
import numpy as np
import PIL.Image
from io import BytesIO
import IPython.display
import ipywidgets as widgets
import threading

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
#Use 'jpeg' instead of 'png' (~5 times faster)
def showarray(a, prev_display_id=None, fmt='jpeg'):
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    obj = IPython.display.Image(data=f.getvalue())
    if prev_display_id is not None:
        IPython.display.update_display(obj, display_id=prev_display_id)
        return prev_display_id
    else:
        return IPython.display.display(obj, display_id=True)

In [4]:
def get_frame(cam):
    # Capture frame-by-frame
    ret, frame = cam.read()
    
    #flip image for natural viewing
#     frame = cv2.flip(frame, 1)
    
    return frame

In [5]:
cameras = []

def init_cameras():
    
    for usb_camera in usb_cameras:

        cam = cv2.VideoCapture(usb_camera.get('path'))

#         cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
#         cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
        cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
        cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
        
#         cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
#         cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

#         cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
#         cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)


    #     cam.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25) #ref: https://github.com/opencv/opencv/issues/9738#issuecomment-346584044
    #     cam.set(cv2.CAP_PROP_EXPOSURE, 0.01)
    #     cam.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0)
    #     cam.set(cv2.CAP_PROP_EXPOSURE, -4.0)

        cameras.append({
            "name": usb_camera.get('name'),
            'cam': cam,
            'display_id': None,
            "offset":  usb_camera.get('offset'),
        })
        
def stop_cameras():      
    for camera in cameras:
        cam = camera.get('cam')
        if(cam):
            cam.release()
        
def test_cams(usb_cameras):
    for usb_camera in usb_cameras:
#         print(index)
        cap = cv2.VideoCapture()
        cap.open(usb_camera.get('path'))
        if cap.isOpened():
            print("active: ", usb_camera.get('path'))
        else:
            print("inactive: ", usb_camera.get('path'))
        cap.release()

In [6]:
usb_cameras = [
#     {
#         "path": "/dev/zuppacamera5",
#         "name": "zuppacamera5",
#         "offset": 0.56,
#     },
#     {
#         "path": "/dev/zuppacamera4",
#         "name": "zuppacamera4",
#         "offset": 0.45,
#     },
    {
        "path": "/dev/zuppacamera3",
        "name": "zuppacamera3",
        "offset": 0.56,
    },
    {
        "path": "/dev/zuppacamera2",
        "name": "zuppacamera2",
        "offset": 0.44,
    },
    {
        "path": "/dev/zuppacamera1",
        "name": "zuppacamera1",
        "offset": 0.46,
    },
    {
#         "path": "/dev/zuppacamera0", #cannot use zuppacamera0 due to bandwidth issue with zuppacamera1
        "path": "/dev/zuppacamera5",
        "name": "zuppacamera0",
        "offset": 0.54,
    },
]

print(usb_cameras)
test_cams(usb_cameras)

[{'path': '/dev/zuppacamera3', 'name': 'zuppacamera3', 'offset': 0.56}, {'path': '/dev/zuppacamera2', 'name': 'zuppacamera2', 'offset': 0.44}, {'path': '/dev/zuppacamera1', 'name': 'zuppacamera1', 'offset': 0.46}, {'path': '/dev/zuppacamera5', 'name': 'zuppacamera0', 'offset': 0.54}]
active:  /dev/zuppacamera3
active:  /dev/zuppacamera2
active:  /dev/zuppacamera1
active:  /dev/zuppacamera5


In [7]:
stop_cameras()
init_cameras()

In [8]:
from datetime import datetime
import os

button = widgets.Button(description="Capture All")
button_output = widgets.Output()

display(button, button_output)

main_dir = './captured'
sub_dir = '/tests'
final_dir = main_dir+sub_dir

if not os.path.exists(final_dir):
    os.makedirs(final_dir)

def on_button_clicked(b):
    with button_output:
        
#         print("Capturing...")
#         IPython.display.clear_output(wait=True)

        datetime_str = datetime.today().strftime('%Y-%m-%d_%H:%M:%S')
    
        for camera_num, camera in enumerate(cameras):

            name = camera.get('name')
            cam = camera.get('cam')
            for x in range(10): #lame way to clear image buffer
                frame = get_frame(cam)

            #crop start
            offset = camera.get('offset')
            image = frame
            width = image.shape[1]
            height = image.shape[0]
#             print('width', width)
#             print('height', height)
#             print('offset', offset)

            y_half = int((height-1)*offset) #cut of 50% of from top

            image_areas = []

            # top half
            image_areas.append({
                "name": "t",
                "y1": 0,
                "y2": y_half,
                "x1": 0,
                "x2": width-1,
                "rotate": True,
            })

            # bottom half
            image_areas.append({
                "name": "b",
                "y1": y_half,
                "y2": height,
                "x1": 0,
                "x2": width-1,
                "rotate": False,
            })


            for image_area in image_areas:

                image_crop = image[image_area.get('y1'):image_area.get('y2'), image_area.get('x1'):image_area.get('x2')]

                save_image = image_crop
                
                ENABLE_ROTATE = False
                ENABLE_ROTATE = True
                
                if(ENABLE_ROTATE and image_area.get('rotate') is True):
                    #ref: https://www.tutorialkart.com/opencv/python/opencv-python-rotate-image/
                    (h, w) = image_crop.shape[:2]
                    # calculate the center of the image
                    center = (w / 2, h / 2)

                    M = cv2.getRotationMatrix2D(center, 180, 1.0)
                    save_image = cv2.warpAffine(image_crop, M, (w, h))

                final_name = datetime_str+'_camera_'+str(name)+'_'+image_area.get('name')+'.jpg'
                cv2.imwrite(final_dir + '/' + final_name, save_image) #need to create folder called captures first!

                image_crop = cv2.cvtColor(image_crop, cv2.COLOR_BGR2RGB)
                showarray(image_crop)
            
        print("Done")

        IPython.display.clear_output(wait=True)
        

button.on_click(on_button_clicked)

Button(description='Capture All', style=ButtonStyle())

Output()

In [9]:

# stop_cameras()        
    
# print ("Stream stopped")